# Medzy
## Overview
This project aims to develop a machine learning model capable of interpreting doctors’ handwriting on prescriptions. By accurately detecting and translating challenging handwriting, the model will empower patients to read their prescriptions independently, making it easier for them to purchase their medications without confusion if they run out of medicine.

This model is using Hugging Face's [BART model](https://huggingface.co/docs/transformers/en/model_doc/bart) for correcting the OCR errors.

## Get PyTorch Device

### DirectML

In [2]:
import torch
import torch_directml

device = torch_directml.device()

ModuleNotFoundError: No module named 'torch_directml'

### CUDA (fallback to CPU if none)

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading the labels

In [4]:
import pandas as pd

train_df = pd.read_csv("./Dataset/Training/training_labels.csv", delimiter = ",")
labels = train_df["MEDICINE_NAME"].unique() # Get all unique instance of the labels

## OCR Errors Simulation

In [ ]:
import re
import random

# Define common OCR misread patterns
ocr_confusions = {
    '0': 'O', 'O': '0', '1': 'I', 'I': '1', '5': 'S', 'S': '5',
    'rn': 'm', 'm': 'rn', 'l': 'I', 'I': 'l', 'B': '8', '8': 'B',
    'c': 'e', 'e': 'c', 'a': 'o', 'o': 'a', 'e': 'B'
}

def reduce_repetitions(word):
    """Reduces repeated patterns like 'Arolrolrol' to 'Arol'."""
    return re.sub(r'(.{2,})\1+', r'\1', word)

def introduce_ocr_typo(word):
    """Simulates common OCR errors in a word."""
    typo_types = ["swap", "delete", "insert", "replace", "ocr_confusion", "repeat"]
    typo = random.choice(typo_types)
    index = random.randint(0, len(word) - 1)

    if typo == "swap" and index < len(word) - 1:
        word = word[:index] + word[index+1] + word[index] + word[index+2:]
    elif typo == "delete":
        word = word[:index] + word[index+1:]
    elif typo == "insert":
        random_char = random.choice("abcdefghijklmnopqrstuvwxyz")
        word = word[:index] + random_char + word[index:]
    elif typo == "replace":
        random_char = random.choice("abcdefghijklmnopqrstuvwxyz")
        word = word[:index] + random_char + word[index+1:]
    elif typo == "ocr_confusion":
        for key, val in ocr_confusions.items():
            if key in word:
                word = word.replace(key, val, 1)  # Replace one occurrence
                break
    elif typo == "repeat":
        word = word[:index] + word[index:] + word[index:]  # Repeat part of the word

    return reduce_repetitions(word)  # Fix excessive repetition

In [6]:
# Generate Dataset with OCR distortions
dataset = [{"input": introduce_ocr_typo(label), "target": label} for label in labels for _ in range(5)]

In [7]:
dataset

[{'input': 'ceta', 'target': 'Aceta'},
 {'input': 'Aceta', 'target': 'Aceta'},
 {'input': 'Actea', 'target': 'Aceta'},
 {'input': 'oceta', 'target': 'Aceta'},
 {'input': 'Aeeta', 'target': 'Aceta'},
 {'input': 'Aee', 'target': 'Ace'},
 {'input': 'Aee', 'target': 'Ace'},
 {'input': 'Aee', 'target': 'Ace'},
 {'input': 'Ace', 'target': 'Ace'},
 {'input': 'Ace', 'target': 'Ace'},
 {'input': 'Alatroll', 'target': 'Alatrol'},
 {'input': 'Alatrom', 'target': 'Alatrol'},
 {'input': 'Alatrol', 'target': 'Alatrol'},
 {'input': 'AIatrol', 'target': 'Alatrol'},
 {'input': 'Alatryol', 'target': 'Alatrol'},
 {'input': 'Arnodis', 'target': 'Amodis'},
 {'input': 'modis', 'target': 'Amodis'},
 {'input': 'Amodis', 'target': 'Amodis'},
 {'input': 'Amodes', 'target': 'Amodis'},
 {'input': 'Amdis', 'target': 'Amodis'},
 {'input': 'Artizin', 'target': 'Atrizin'},
 {'input': 'Atrizidn', 'target': 'Atrizin'},
 {'input': 'Atrizfin', 'target': 'Atrizin'},
 {'input': 'Atrizin', 'target': 'Atrizin'},
 {'input': '

## Model Training

In [8]:
from transformers import BartTokenizer

# Load tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Tokenization function
def preprocess(example):
    inputs = tokenizer(example["input"], padding="max_length", truncation=True, max_length=32)
    targets = tokenizer(example["target"], padding="max_length", truncation=True, max_length=32)
    
    inputs["labels"] = targets["input_ids"]  # Target labels for training
    return inputs

# Apply preprocessing
tokenized_datasets = list(map(preprocess, dataset))

A data collator dynamically pads sequences to the longest in a batch, reducing unnecessary padding.

In [9]:
from transformers import DataCollatorForSeq2Seq

# Use Hugging Face's built-in data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None, padding=True)

In [ ]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

# Load pre-trained BART model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Training arguments
training_args = TrainingArguments(
    output_dir="./bart_autocorrect",
    eval_strategy="no",
    save_strategy="epoch",
    per_device_train_batch_size=64,  # Adjust batch size depending on your GPU
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=5,
    logging_dir="./logs",
    save_total_limit=2,
    push_to_hub=False
)

# Define Trainer with batching
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    processing_class=tokenizer,
    data_collator=data_collator  # Enables batching
)

In [11]:
# Start training
trainer.train()

Step,Training Loss


e:\CondaEnvs\pyt_cuda\lib\site-packages\transformers\modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=125, training_loss=5.4998134765625, metrics={'train_runtime': 38.3383, 'train_samples_per_second': 50.863, 'train_steps_per_second': 3.26, 'total_flos': 37155815424000.0, 'train_loss': 5.4998134765625, 'epoch': 5.0})

In [12]:
# Save model
model.save_pretrained("./model-output/BART")

# Save tokenizer
tokenizer.save_pretrained("./model-output/BART")

('./model-output/BART\\tokenizer_config.json',
 './model-output/BART\\special_tokens_map.json',
 './model-output/BART\\vocab.json',
 './model-output/BART\\merges.txt',
 './model-output/BART\\added_tokens.json')